In [1]:
import sys
import requests
from bs4 import BeautifulSoup
import re
import unicodedata
import pandas as pd

In [21]:
def date_time(table_cells):
    return [data_time.strip() for data_time in list(table_cells.strings)][0:2]

def booster_version(table_cells):
    out=''.join([booster_version for i,booster_version in enumerate( table_cells.strings) if i%2==0][0:-1])
    return out

def landing_status(table_cells):
    out=[i for i in table_cells.strings][0]
    return out


def get_mass(table_cells):
    mass=unicodedata.normalize("NFKD", table_cells.text).strip()
    if mass:
        mass.find("kg")
        new_mass=mass[0:mass.find("kg")+2]
    else:
        new_mass=0
    return new_mass


def extract_column_from_header(row):
    if (row.br):
        row.br.extract()
    if row.a:
        row.a.extract()
    if row.sup:
        row.sup.extract()    
    colunm_name = ' '.join(row.contents)
    if not(colunm_name.strip().isdigit()):
        colunm_name = colunm_name.strip()
        return colunm_name

In [9]:
# Old URL version: 9th June 2021
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"
response = requests.get(static_url)
reader = BeautifulSoup(response.text, 'html.parser')
print(reader.title)

<title>List of Falcon 9 and Falcon Heavy launches - Wikipedia</title>


In [22]:
html_tables = reader.find_all('table')
table = html_tables[2]

column_names = []
th_el = table.find_all('th')
for i in th_el:
    name = extract_column_from_header(i)
    if name is not None and len(name) > 0:
        column_names.append(name)

print(column_names)

['Flight No.', 'Date and time ( )', 'Launch site', 'Payload', 'Payload mass', 'Orbit', 'Customer', 'Launch outcome']


In [23]:
launch_dict= dict.fromkeys(column_names)
del launch_dict['Date and time ( )']

launch_dict['Flight No.'] = []
launch_dict['Launch site'] = []
launch_dict['Payload'] = []
launch_dict['Payload mass'] = []
launch_dict['Orbit'] = []
launch_dict['Customer'] = []
launch_dict['Launch outcome'] = []
launch_dict['Version Booster']=[]
launch_dict['Booster landing']=[]
launch_dict['Date']=[]
launch_dict['Time']=[]

In [ ]:
extracted_row = 0
for table_number,table in enumerate(reader.find_all('table',"wikitable plainrowheaders collapsible")):
    for rows in table.find_all("tr"):
        if rows.th:
            if rows.th.string:
                flight_number=rows.th.string.strip()
                flag=flight_number.isdigit()
        else:
            flag=False
        row=rows.find_all('td')
        if flag:
            extracted_row += 1
            launch_dict["Flight No."].append(flight_number)
            datatimelist=date_time(row[0])
            date = datatimelist[0].strip(',')
            launch_dict["Date"].append(date)
            time = datatimelist[1]
            launch_dict["Time"].append(time)
            bv=booster_version(row[1])
            if not(bv):
                bv=row[1].a.string
            launch_dict["Version Booster"].append(bv)
            launch_site = row[2].a.string
            launch_dict["Launch site"].append(launch_site)
            payload = row[3].a.string
            launch_dict["Payload"].append(payload)
            payload_mass = get_mass(row[4])
            launch_dict["Payload mass"].append(payload_mass)
            orbit = row[5].a.string
            launch_dict["Orbit"].append(orbit)
            customer = row[6].a.strings
            launch_dict["Customer"].append(customer)
            launch_outcome = list(row[7].strings)[0]
            launch_dict["Launch outcome"].append(launch_outcome)
            booster_landing = landing_status(row[8])
            launch_dict["Booster landing"].append(booster_landing)

headings = []
for key,values in dict(launch_dict).items():
    if key not in headings:
        headings.append(key)
    if values is None:
        del launch_dict[key]

lmax = 0
for lname in launch_dict.keys():
    lmax = max(lmax, len(launch_dict[lname]))
for lname in launch_dict.keys():
    ll = len(launch_dict[lname])
    if  ll < lmax:
        launch_dict[lname] += [0] * (lmax - ll)

df = pd.DataFrame.from_dict(launch_dict)

In [41]:
df

,Flight No.,Launch site,Payload,Payload mass,Orbit,Customer,Launch outcome,Version Booster,Booster landing,Date,Time
0,1,CCAFS,Dragon Spacecraft Qualification Unit,0,LEO,SpaceX,Success\n,F9 v1.0B0003.1,Failure,4 June 2010,18:45
1,2,CCAFS,Dragon,0,LEO,NASA,Success,F9 v1.0B0004.1,Failure,8 December 2010,15:43
2,3,CCAFS,Dragon,525 kg,LEO,NASA,Success,F9 v1.0B0005.1,No attempt\n,22 May 2012,07:44
3,4,CCAFS,SpaceX CRS-1,"4,700 kg",LEO,NASA,Success\n,F9 v1.0B0006.1,No attempt,8 October 2012,00:35
4,5,CCAFS,SpaceX CRS-2,"4,877 kg",LEO,NASA,Success\n,F9 v1.0B0007.1,No attempt\n,1 March 2013,15:10
...,...,...,...,...,...,...,...,...,...,...,...
571,102,CCSFS,SXM-7,"7,000 kg",GTO,<generator object Tag._all_strings at 0x000001...,Success\n,F9 B5 ♺,Success,13 December 2020,17:30:00
572,103,KSC,NROL-108,C,LEO,<generator object Tag._all_strings at 0x000001...,Success\n,F9 B5 ♺,Success,19 December 2020,14:00:00
573,104,CCSFS,Türksat 5A,"3,500 kg",GTO,0,0,F9 B5,0,8 January 2021,02:15
574,105,KSC,Starlink,"15,600 kg",LEO,0,0,F9 B5B1051.8,0,20 January 2021,13:02


In [42]:
df.to_csv('spacex_web_scraped.csv', index=False)